In [1]:
! pip install -U transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/b6/4d/fbe6d89fde59d8107f0a02816c4ac4542a8f9a85559fdf33c68282affcc1/transformers-4.38.2-py3-none-any.whl.metadata
  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from https://files.pythonhosted.org/packages/ab/28/d4b691840d73126d4c9845f8a22dad033ac872509b6d3a0d93b456eef424/huggingface_hub-0.21.4-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.21.4-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/eb/10/4ccc8eed80f11c082a2883d49d4090aa80c7f65704216a529f490cb089b1/regex-2023.12.25-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached regex-2023.12.25-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Obtaining dependency information for tokenizers<0.19,>=0.14

In [2]:
! pip install datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/95/fc/661a7f06e8b7d48fcbd3f55423b7ff1ac3ce59526f146fda87a1e1788ee4/datasets-2.18.0-py3-none-any.whl.metadata
  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
  Obtaining dependency information for pyarrow>=12.0.0 from https://files.pythonhosted.org/packages/cc/58/086a8c7e8b4c8ee9f505eec227a7641e385c06a2ac9d4bbf18447b4e1ed2/pyarrow-15.0.1-cp38-cp38-manylinux_2_28_x86_64.whl.metadata
  Using cached pyarrow-15.0.1-cp38-cp38-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/ad/80/8fc9a4d76b259c901f2c85ed10f330a8fb51993a577bd

In [3]:
from datasets import load_dataset 

dataset = load_dataset("eduvedras/Img_Vars", split="train")

/opt/conda/envs/sagemaker-distribution/lib/python3.8/site-packages/datasets/load.py:1461: FutureWarning: The repository for eduvedras/Img_Vars contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eduvedras/Img_Vars
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
from torch.utils.data import Dataset, DataLoader

MAX_PATCHES = 1024

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["Chart"], return_tensors="pt", add_special_tokens=True, max_patches=MAX_PATCHES)
        
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        encoding["Description"] = item["Description"]
        return encoding

In [5]:
from transformers import AutoProcessor, Pix2StructForConditionalGeneration
model_id = "google/pix2struct-textcaps-base"
processor = AutoProcessor.from_pretrained(model_id)
model = Pix2StructForConditionalGeneration.from_pretrained(model_id)

In [6]:
def collator(batch):
  new_batch = {"flattened_patches":[], "attention_mask":[]}
  texts = [item["Description"] for item in batch]
  
  text_inputs = processor(text=texts, padding="max_length", return_tensors="pt", add_special_tokens=True, max_length=166)
  
  new_batch["labels"] = text_inputs.input_ids
  
  for item in batch:
    new_batch["flattened_patches"].append(item["flattened_patches"])
    new_batch["attention_mask"].append(item["attention_mask"])
  
  new_batch["flattened_patches"] = torch.stack(new_batch["flattened_patches"])
  new_batch["attention_mask"] = torch.stack(new_batch["attention_mask"])

  return new_batch

In [7]:
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2, collate_fn=collator)

In [ ]:
import torch
import huggingface_hub
from tqdm import tqdm

huggingface_hub.login('',True,False,False)
model_name = model_id.split("/")[1]

EPOCHS = 63

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()

for epoch in tqdm(range(EPOCHS)):
    print("Epoch:", epoch)
    if (epoch + 1) % 20 == 0:
        model.push_to_hub(f"{model_name}-desc-vars-final")
    for idx, batch in enumerate(train_dataloader):
        labels = batch.pop("labels").to(device)
        flattened_patches = batch.pop("flattened_patches").to(device)
        attention_mask = batch.pop("attention_mask").to(device)

        outputs = model(flattened_patches=flattened_patches,
                        attention_mask=attention_mask,
                        labels=labels)

        loss = outputs.loss

        print("Loss:", loss.item())

        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        if (epoch + 1) % 20 == 0:
            model.eval()

            predictions = model.generate(flattened_patches=flattened_patches, attention_mask=attention_mask)        
            print("Predictions:", processor.batch_decode(predictions, skip_special_tokens=True))




Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /home/studio-lab-user/.cache/huggingface/token
Login successful


  0%|          | 0/63 [00:00<?, ?it/s]

Epoch: 0
Loss: 27.539587020874023
Loss: 26.689891815185547
Loss: 28.286235809326172
Loss: 21.287620544433594
Loss: 24.557132720947266
Loss: 21.823354721069336
Loss: 25.669546127319336
Loss: 24.576026916503906
Loss: 22.962800979614258
Loss: 25.834272384643555
Loss: 25.072450637817383
Loss: 23.41463279724121
Loss: 22.915016174316406
Loss: 22.870162963867188
Loss: 23.773395538330078
Loss: 23.415298461914062
Loss: 21.443838119506836
Loss: 21.514514923095703
Loss: 21.064828872680664
Loss: 21.76187515258789
Loss: 19.642311096191406
Loss: 19.69300651550293
Loss: 19.848854064941406
Loss: 20.894939422607422
Loss: 19.127105712890625
Loss: 19.688093185424805
Loss: 16.032325744628906
Loss: 16.332115173339844
Loss: 17.446090698242188
Loss: 16.651466369628906
Loss: 17.373544692993164
Loss: 17.421518325805664
Loss: 17.924711227416992
Loss: 15.474569320678711
Loss: 17.50925636291504
Loss: 16.92034912109375
Loss: 15.217510223388672
Loss: 17.391155242919922
Loss: 16.032602310180664
Loss: 17.054828643798

  2%|▏         | 1/63 [01:29<1:32:07, 89.16s/it]

Loss: 6.187178134918213
Epoch: 1
Loss: 5.479941368103027
Loss: 5.531160354614258
Loss: 5.536831378936768
Loss: 6.252987861633301
Loss: 5.694084644317627
Loss: 5.377201080322266
Loss: 5.2012619972229
Loss: 5.518664836883545
Loss: 6.120620250701904
Loss: 5.592771530151367
Loss: 5.926486015319824
Loss: 5.619228363037109
Loss: 6.0750885009765625
Loss: 5.808279037475586
Loss: 6.12955904006958
Loss: 5.972214221954346
Loss: 5.694229602813721
Loss: 6.222742080688477
Loss: 5.537179470062256
Loss: 5.915146350860596
Loss: 5.841073513031006
Loss: 5.9164204597473145
Loss: 5.891717910766602
Loss: 4.693178176879883
Loss: 5.447741508483887
Loss: 5.5254316329956055
Loss: 5.878856182098389
Loss: 6.030336856842041
Loss: 6.035490036010742
Loss: 5.628006935119629
Loss: 5.575549602508545
Loss: 5.188356876373291
Loss: 6.102481842041016
Loss: 5.745399475097656
Loss: 5.154699325561523
Loss: 5.904969692230225
Loss: 5.5325775146484375
Loss: 5.735956192016602
Loss: 5.464939117431641
Loss: 5.4704132080078125
Loss:

...

Loss: 0.0009978885063901544


/opt/conda/envs/sagemaker-distribution/lib/python3.8/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predictions: ['Gender', 'hours-per-week <= 41.5, capital-loss <= 18']
Loss: 0.000896371784619987
Predictions: ['ssc_p, hsc_p, degree_p, etest_p', 'Air temperature [K], Process temperature [K], Rotational speed [rpm], Torque']
Loss: 0.001033353852108121
Predictions: ['3', 'ReachedOnTime']
Loss: 0.0010253289947286248
Predictions: ['4', '8']
Loss: 0.000823836016934365
Predictions: ['age, height, weight, waistline, SBP, BLDS, tot_c', 'SMK_stat_type_cd <= 1.5, gamma_GTP <=']
Loss: 0.0009022348094731569
Predictions: ['Pregnancies, Glucose, BloodPressure, SkinThickness, In', 'basic_needs <= 3.5, bullying <= 1.5']
Loss: 0.0009474579128436744
Predictions: ['JoiningYear, PaymentTier, Age, ExperienceInCurrentDomain', 'skewness <= 5.16, curtosis <= 0.1']
Loss: 0.0008678978192619979
Predictions: ['Alcohol, Malic acid, Ash, Alcalinity of ash, Total phenol', '6']
Loss: 0.0009705916163511574
Predictions: ['age, fnlwgt, educational-num, capital-gain, capital-loss, hours', '7']
Loss: 0.00092901138123124

 94%|█████████▎| 59/63 [1:32:03<06:14, 93.62s/it]


KeyboardInterrupt: 